<a href="https://colab.research.google.com/github/JoaoMMS98/ML_2.0/blob/main/Chiel_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Model Selection

In this notebook, we are going to apply different ways to split your dataset and try to select the best model.

__`Step 1`__ Import the needed libraries

In [29]:
!pip install xgboost==1.7.5
!pip install scikit-learn==1.2.2

In [34]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Perceptron
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

Chiel > XGBoost,CatBoost.

Mariana > KNN, RFM, SVM

In [38]:
# Import the data
train_df = pd.read_csv('train_data1.csv')
test_df = pd.read_csv('test_data1.csv')

In [42]:
train_df.select_dtypes(include=np.number)

,Alternative Dispute Resolution,Attorney/Representative,COVID-19 Indicator,Zip Code,Agreement Reached,First Hearing Month,First Hearing Year,Gender_F,Gender_M,Gender_U,Gender_X,Age at Injury,Average Weekly Wage,Birth Year,IME-4 Count,Industry Code,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Number of Dependents,log_Average Weekly Wage,fe_Carrier Type,fe_Medical Fee Region,fe_County of Injury,Age,Accident Month,Accident Year,C-2 Month,C-2 Year,C-3 Month,C-3 Year,Assembly Month,Assembly Year,fe_Industry Code,fe_WCIO Part Of Body Code,fe_WCIO Cause of Injury Code,fe_Zip Code,fe_WCIO Nature of Injury Code,Claim Injury Type
0,0,0,0,10451,0,4,2022,1.0,0.0,0.0,0.0,-0.913043,0.000000,1.045455,0.0,0.647059,-0.909091,-0.333333,-0.15,0.25,0.0000,0.937419,0.718864,0.233954,1.045455,0.833333,0.0,1.000000,0.0,0.0,0.0,0.833333,0.0,-0.231530,0.174332,-0.324486,0.482007,-0.190844,2
1,0,0,0,14620,0,4,2022,1.0,0.0,0.0,0.0,-0.217391,0.000000,0.227273,0.0,1.823529,0.022727,0.111111,-0.35,0.50,0.0000,0.000000,-0.281136,-0.145258,0.227273,-0.333333,-0.5,-0.166667,-0.5,3.0,-1.0,-0.333333,-0.5,0.652579,-0.374483,0.448708,-0.257361,0.642015,2
2,0,1,0,10455,0,4,2022,0.0,1.0,0.0,0.0,-0.521739,0.000000,0.590909,0.0,-2.235294,0.000000,0.000000,0.00,-0.25,0.0000,-0.691407,0.718864,0.233954,0.590909,-1.000000,0.0,-0.166667,0.0,-2.0,0.0,-0.833333,0.0,-0.172972,0.816401,2.089949,0.149400,0.000000,2
3,0,1,0,14226,0,2,2023,0.0,1.0,0.0,0.0,1.565217,0.000000,-1.545455,0.0,1.823529,-0.568182,-0.444444,-2.35,0.25,0.0000,0.000000,-0.281136,0.021994,-1.545455,-0.166667,0.5,0.166667,0.5,3.0,1.0,0.000000,0.5,0.652579,0.468693,0.687572,-0.129771,-0.461531,3
4,0,1,0,10550,0,4,2023,1.0,0.0,0.0,0.0,0.086957,0.000000,0.000000,-1.0,0.058824,0.409091,0.000000,0.20,-0.50,0.0000,-0.062581,-0.453044,-0.100729,0.000000,-0.500000,0.5,0.666667,0.5,6.0,1.0,0.500000,0.5,0.946111,0.738707,0.937399,0.524537,0.000000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459215,0,0,0,13797,0,1,2022,0.0,1.0,0.0,0.0,0.565217,0.000000,-0.500000,0.0,-1.000000,0.931818,-0.444444,-0.15,0.75,0.0000,-0.398175,0.000000,-0.558616,-0.500000,0.500000,0.0,1.000000,0.0,0.0,0.0,0.833333,0.0,0.000000,0.174332,-0.236499,-0.678299,-0.461531,2
459216,0,0,1,10451,0,4,2022,0.0,1.0,0.0,0.0,-0.739130,0.000000,0.818182,0.0,0.588235,0.613636,1.259259,2.65,-0.50,0.0000,0.937419,0.718864,0.233954,0.818182,0.500000,-0.5,0.833333,-0.5,0.0,0.0,0.666667,-0.5,-0.453838,-0.379886,0.375606,0.482007,-0.357985,2
459217,0,0,0,11722,0,4,2022,1.0,0.0,0.0,0.0,-0.652174,0.000000,0.681818,0.0,0.058824,0.977273,0.111111,0.20,-0.25,0.0000,0.937419,0.718864,0.731629,0.681818,0.166667,-0.5,0.666667,-0.5,5.0,-1.0,0.333333,-0.5,0.946111,0.738707,0.255423,0.862595,0.642015,3
459218,0,0,0,12586,0,4,2022,1.0,0.0,0.0,0.0,0.826087,2.947265,-0.772727,0.0,0.058824,-0.613636,0.000000,0.90,-0.25,1.1617,0.937419,-0.281136,-0.300387,-0.772727,-0.500000,0.5,-0.333333,0.5,1.0,1.0,-0.500000,0.5,0.946111,-0.168177,0.577025,-0.203926,0.000000,4


## 1.1. The train-test split

8:2 splitting target column

In [43]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Assuming 'train_df' is your DataFrame
data = train_df.drop('Claim Injury Type', axis=1)  # Independent variables
target = train_df['Claim Injury Type']  # Target variable

# Split the data
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=15, shuffle=True, stratify=target)

# Label encode the target variable
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

print('train:{}% | test:{}%'.format(round(len(y_train)/len(target),2),
                                                     round(len(y_test)/len(target),2)))

pd.set_option('display.max_columns', None)  # Display all columns (optional)

train:0.8% | test:0.2%


Stratified Sampling: Ensures that the training and testing sets maintain the same class distribution as the original dataset, reducing bias.


Repeated Hold-Out: The process is repeated multiple times with different splits, and the average performance is considered.

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:Carrier Name: object, Carrier Type: object, County of Injury: object, District Name: object, Gender: object, Medical Fee Region: object, WCB Decision: object